<a href="https://colab.research.google.com/github/jenskuehne-cmd/Colab_Skripte_OEE/blob/main/Notebooks/RoleMappingAspire/NichtRMVerteilt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Beschreibung des Notebooks

Dieses Notebook dient dazu, Benutzerdaten und deren zugewiesene Rollen aus verschiedenen Google Sheets zu sammeln, zu verarbeiten und zusammenzuführen. Es hat zwei Hauptziele:

1.  **Identifikation von Benutzern ohne zugewiesene Rollen**: Es filtert und exportiert eine Liste von Benutzern, denen bestimmte Rollen in einem System (EAM) fehlen, in eine CSV-Datei (`User_Ohne_RM.csv`). Dies hilft dabei, Benutzer zu identifizieren, die möglicherweise keine ausreichenden Berechtigungen haben.

2.  **Zusammenführung und Analyse von Benutzerrollen**: Es lädt detaillierte Benutzer- und Rolleninformationen aus drei verschiedenen Google Sheets (KAU, PTD und PS). Die verschiedenen Rollentypen (`P30 Rollen Asset`, `Maintenance KAU Roles`, `EAM Roles`, etc.) werden extrahiert und in einer konsolidierten Spalte (`S4H_Roles`) zusammengeführt. Diese konsolidierten Rollen werden dann mit den grundlegenden Benutzerinformationen wie `UserID`, `Name`, `Vorname` etc. kombiniert, um eine umfassende Sicht auf alle Benutzer und deren Rollenzuweisungen für weitere Analysen zu ermöglichen.

In [ ]:
# @title
%%capture
pip install gspread==5.11.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2


In [ ]:
# @title
%%capture
pip install google-auth==2.23.3

In [ ]:
# @title
from google.colab import auth
auth.authenticate_user()

Dieser code exportiert eine CSV Tabelle die alle USER ANzeigt die erstaunlicherweise keine Rollen zugeteilt bekommen haben im EAM

In [ ]:
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung für Google Drive und Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Hier die Sheet ID anstatt der URL einfügen
sheet_id = "1HFw-jrF7e0b1je4vseA1V5gLDKuwRpoI3vcttWhhXaQ"
worksheet_name = "RM_KAU"

# Google Sheet mit der Sheet ID öffnen
sheet = gc.open_by_key(sheet_id).worksheet(worksheet_name)
df = pd.DataFrame(sheet.get_all_records())

# Filter the DataFrame based on the specified criteria
User_Ohne_RM = df[
    (df["Mapping erledigt"] != "TRUE") &
    (df["P30 Rollen Asset"].notna()) &
    (
        (df["Maintenance KAU Roles"].isna()) |
        (df["Maintenance KAU Roles"] == "1201_Maintenance Requester_KAU") |
        (df["Maintenance KAU Roles"] == "1201_General Assets Display")
    )
]
# Save the filtered DataFrame to a CSV file
User_Ohne_RM.to_csv("User_Ohne_RM.csv", index=False)

print("Successfully created 'User_Ohne_RM.csv'")

Successfully created 'User_Ohne_RM.csv'


In [ ]:
from google.colab import files
files.download('User_Ohne_RM.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
PD

UserID,User - User ID,Name,Vorname,User - Organization,Manager Name,Manager Email,BMRAM Rollen bestehend,
User - Workday Position Title,User - Organization - Supervisory Organization Short Code,
User - Location,User - Cost Center Reference,User - Cost Center,EAM Roles

PS
UserID,UserNr,Name,Vorname,User - Organization,Circle,Circle Lead,No Aspire Role needed,Alle P30 Rollen (ausser Asset),
P30 Asset Rollen,User - Workday Position Title,Circles PS,Scopes,Abteilung,ASSET EAM Role,Manager Name,Manager Email,
User - Location,User - Cost Center Reference,Maintenance,Kalibrierung,MDG EAM,MDG MM,Analyse Reporting,
Ersatzteilmanagement,Engineering,Engineering - Capex,MA Rollen zugeteilt Assets,MA zugeteilt Engineering

KAU
Mapping erledigt,UserID,User - User ID,Name,Vorname,User - Organization,P30 Rollen Asset,P30 alle,User - Workday Position Title,WC,WC Name,Organization,
Manager Name,Manager Email,User - Location,Abgleich P30 Rolle,Höhere Rolle als Display?,No Aspire Role needed,
Main Workstream,RM SPOC (USERID),RM SPOC2 (USERID),Cross functional Role-Mapping needed? Workstream,
Workstream(s) Cross Functional,RM-SPOC(s) for Crossfunctional Role-Mapping,Comments,aufgenommen in RM-Tool,Plausi Check,
Maintenance KAU Roles,Analyse Reporting,Ersatzteilmanagem.,Kalibrierung,EWM KAU,MDG EAM,MDG MM

In [ ]:
# @title
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np

# Authenticate to Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# KAU DATEN
# Open the Google Sheet
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit?gid=2009282802#gid=2009282802'
sheet = gc.open_by_url(spreadsheet_url)
worksheet = sheet.get_worksheet(0)

# Load data into Pandas DataFrame
data_KAU = pd.DataFrame(worksheet.get_all_records()) # Changed DataFrame name to data_KAU

# PTD DATEN
# Open the Google Sheet for PTD
spreadsheet_url_ptd = 'https://docs.google.com/spreadsheets/d/1wguV7OQ-0VlNLO5PTFucD4mPnNiCdwzRmpSZA33rL1I/edit?gid=1165668754#gid=1165668754'
sheet_ptd = gc.open_by_url(spreadsheet_url_ptd)
worksheet_ptd = sheet_ptd.worksheet("PTD User") # Access the "PTD User" sheet

# PTD Load data into Pandas DataFrame
data_ptd = pd.DataFrame(worksheet_ptd.get_all_records())
# 1. Extract roles from data_ptd
ptd_roles = []
for role in data_ptd['EAM Roles']:  # Extract from 'EAM Roles' column
    if isinstance(role, str):  # Check if role is a string
        roles_in_cell = role.split(',')  # Split by comma
        for r in roles_in_cell:
            cleaned_role = r.strip()
            if cleaned_role and cleaned_role.lower() != 'nan' and cleaned_role != '':
                ptd_roles.append(cleaned_role)

# PS DATEN
# Open the Google Sheet for PS
spreadsheet_url_ps = 'https://docs.google.com/spreadsheets/d/13ZOtTr2yKRPZnzsql8C8DOFaFzY8BXaculHtYliilhY/edit?pli=1&gid=734168569#gid=734168569'
sheet_ps = gc.open_by_url(spreadsheet_url_ps)
worksheet_ps = sheet_ps.worksheet("EndUserPS_Unique") # Access the "EndUserPS_Unique" sheet

# Load data into Pandas DataFrame
data_ps = pd.DataFrame(worksheet_ps.get_all_records())

# 2. Extract roles from data_ps
ps_role_columns = ['Maintenance', 'Kalibrierung', 'MDG EAM', 'MDG MM', 'Analyse Reporting',
                   'Ersatzteilmanagement', 'Engineering', 'Engineering - Capex']
ps_roles = []
for index, row in data_ps.iterrows():
    for col in ps_role_columns:
        role = row[col]  # Get role from the current column
        if isinstance(role, str):  # Check if role is a string
            roles_in_cell = role.split(',')  # Split by comma
            for r in roles_in_cell:
                cleaned_role = r.strip()
                if cleaned_role and cleaned_role.lower() != 'nan' and cleaned_role != '':
                    ps_roles.append(cleaned_role)


# KAU Define the specific columns to use for roles
role_columns = ['Maintenance KAU Roles', 'Analyse Reporting', 'Ersatzteilmanagem.', 'Kalibrierung', 'EWM KAU', 'MDG EAM', 'MDG MM']

# Combine roles from the specified columns
all_roles = []

# Iterate over the rows in the DataFrame
for index, row in data_KAU.iterrows():
    for col in role_columns:  # Use the defined columns
        if col in data_KAU.columns:  # Check if the column exists in the DataFrame
            role = row[col]
            if isinstance(role, str):
                roles_in_cell = role.split(',')
                for r in roles_in_cell:
                    cleaned_role = r.strip()
                    if cleaned_role and cleaned_role.lower() != 'nan' and cleaned_role != '':
                        all_roles.append(cleaned_role)
ich habe 3 Tabellen die ich zusmamenführen möchte. Die Rollen welche in dem Datafram ps_roles in den Spalten aus ps_role_columns, data frame data_KAU mit den Spalten aus role_columns und dataframe data_ptd und nur einer Spalte 'EAM Roles' in den verschiedenen Tabellen
stehen sollen in eine Spalte "S4H_Roles" zusammengelegt werden und dann mit den anderen Daten in den Spalten
['UserID', 'Name', 'Vorname', 'User - Organization', 'User - Workday Position Title', 'User - Location', 'Manager Name'] angereichert werden und in eine Tabelle,
oder Dataframe zusammengefahren werden. damit weiter die daten analysiert und gefiltert werden können.

In [ ]:
UserID
Name
Vorname
User - Organization
User - Workday Position Title
User - Location
Manager Name